Lab #4:  Banking Simulation Part I
===================
Submit completed file to Blackboard by 28-OCT-2015.

Create a file named "Lab4.py" containing a function for each problem listed below. Do basic testing of your functions from the IDLE REPL.  Then make sure the tests in this notebook also pass.  When you are sure your code is correct, submit your file to Blackboard.  There is no need to submit the Python notebook file.

<h3>Run this code first:</h3>

In [1]:
import Lab4

<h3>About this lab</h3>

With this lab you will build the framework an *event driven simulation* of bank customers who wait in line before being serviced by a teller.  When your program is completed in next week's lab, you will be able to compare the customer experience when customers wait in a single line or when customers wait in separate teller lines.

<h3>The *event string* data structure</h3>

The basic data structure we will use for keeping track of the state of a customer or a teller during the simulation wlll be called an *event string*.  It is a 18 character string with three format variants:

* *Arrival event:*  An arrival event represents a customer who either has not yet come into the bank, or who is waiting for a teller to be available.  Arrival event strings contain the time of arrival at the bank (# seconds since the bank opened for business) and the service time (# seconds that will be spent with the teller).  Specifically the format looks like this:  'aaaaaA9999999sssss', where aaaaa represents the zero padded arrival time and sssss represents the zero padded service time.  The A identifies that it records an arrival event. 9999999 is a block of placeholder digits which must be present but will be ignored by your code.


* *Service event:*  A service event keeps track of a customer who is either currently being serviced by a teller or has already been serviced and has left the bank.  Service event strings contain the time of arrival and service time (as described above) along with the wait time (# seconds the customer waited in line before being served.  Specifically the format looks like this:  'aaaaaSttwwwwwsssss', , where aaaaa represents the zero padded completion time, tt represents the zero padded teller number, wwwww represents the zero padded wait time, and sssss represents the zero padded service time.  The S identifies that it records a service event.


* *Teller event:*  A teller event keeps track of a teller who is not serving customers.  When a teller doesn't have any customers to serve, the teller will be assigned a random idle time between 0 and 60 seconds before the teller wakes up and checks the queue for new customers.  Teller event strings contain the time for the teller to wake up (# seconds since the bank opened for business) along with the idle time (# seconds).  Specifically the format looks like this:  'aaaaaTttwwwww99999', , where aaaaa represents the zero padded wake up time, tt represents the zero padded teller number, and  wwwww represents the zero padded idle time.  The T identifies that it records an teller event. 9999999 is a block of placeholder digits which must be present but will be ignored by your code.


<h3>The module variable *events*</h3>

You will initialize a module variable called *events* to be an empty list.  This assignment statement should be placed near the top of your file, after any import statements.  *events* is a module variable as opposed to a local variable because it is declared outside your function definitions.  All of your functions will be able to examine its contents and mutate it if necessary.  None of your routines should reassign a new list to it.  

The reason I am asking you to make the "events" list to be defined module wide is because many of the functions you are going to write will need to mutate this list.  If it was not defined module wide, it would have to be passed in as a parameter to nearly every function you write, which is a bit awkward.

The purpose of this variable is to maintain the remaining list of events that need to be processed during the simulation run.  The simulation will start out with the list containing one arrival event string for each customer and one teller event string for each teller (since every teller is going to be idle just before the bank opens).  As the simulation progresses, the events will be removed from this list in order of occurrence and processed.  In some cases new events will be added onto the list as the result of this processing.

<h3>1) Adding new event strings to list *events* </h3>

The customer and teller behavior is completely simulated in this program.  You are going to use Python's random number generator to produce random arrival times and random service times for each customer.  Your code will calculate these times and record them in event strings before it starts simulating the arrival of customers, waiting in teller lines, and servicing of customers.  

Your first step is to write the following three short Python functions that will allow you to create these event strings.  The first two will require you to generate random times, the third does not:

1. A Python function called *makeArrivalEvent* that takes two numeric parameters *maxArrival* and *avgService* representing the maximum arrival time in seconds and the average service time in seconds.  Its job is to create an arrival event string and append it to the *events* list.   The arrival time should be a random int between 0 and maxArrival and the service time should be a random int between 0 and 2*avgService.  

2. A Python function called *makeTellerEvent* that takes two int parameters *currentTime* and *tellerNumber*.  *currentTime* is specified by the number of seconds since the bank opened.  *tellerNumber* is a two digit tell number.  The job of this function is to generate a random idle time (an int between 0 and 60 seconds).  It will then use it to create a new teller event string and append it to the *events* list.  The  wake up time should be the current time + the idle time, the teller number is as specified, and the idle time the one you calculated.

3. A Python function called *makeServiceEvent* that takes four int parameters *currentTime*, *tellerNumber*, *arrivalTime*, and *serviceTime*.  Its job is to create a service event string and append it to the *events* list.  The completion time should be the current time + the service time, the teller number should be as specified, the wait time should be the current time - the arrival time, and the service time should be as specified.

All three functions can be written in 1-4 lines of Python code.

In [2]:
# Test code for your function
import random
random.seed(0)
Lab4.events.clear()
assert(Lab4.makeArrivalEvent( 5000, 100 ) == None)
assert(Lab4.makeArrivalEvent( 2500, 150 ) == None)
assert(Lab4.makeArrivalEvent( 1000, 200 ) == None)
assert(Lab4.makeTellerEvent( 1000, 0 ) == None)
assert(Lab4.makeTellerEvent( 1500, 1 ) == None)
assert(Lab4.makeTellerEvent( 2000, 2 ) == None)
assert(Lab4.makeServiceEvent( 1000, 0, 900, 25 ) == None)
assert(Lab4.makeServiceEvent( 1500, 1, 1000, 50 ) == None)
assert(Lab4.makeServiceEvent( 2000, 2, 1800, 75 ) == None)
assert(len(Lab4.events) == 9)
assert(Lab4.events[0] == '03155A999999900194')
assert(Lab4.events[1] == '01722A999999900020')
assert(Lab4.events[2] == '00265A999999900261')
assert(Lab4.events[3] == '01031T000003199999')
assert(Lab4.events[4] == '01525T010002599999')
assert(Lab4.events[5] == '02058T020005899999')
assert(Lab4.events[6] == '01025S000010000025')
assert(Lab4.events[7] == '01550S010050000050')
assert(Lab4.events[8] == '02075S020020000075')

<h3>2) Breaking out the event fields</h3>

It's important to keep the number of functions that know the details of the event strings to a minimum - this is called *encapsulation*.  That way, if we decide to change the format of event strings, only a small number of functions would need to be changed.

Write the five short and simple functions for extracting each of the fields of an event string:

1. A Python function *getType( event )* which returns the character ('A', 'T', or 'S') that identifies the type of event it represents.

2. A Python function *getTime( event )* which returns as an int the time stored in the first five characters of the event string.

3. A Python function *getTeller( event )* which returns as an int the teller number stored in the event string.

4. A Python function *getWait( event )* which returns as an int the wait time stored in the event string.

5. A Python function *getService( event )* which returns as an int the service time stored in the event string.

Any further code that you write for the bank automation program should use one of these five functions to get information from event strings.  No other code should access the characters of an event string directly.

In [3]:
import random
random.seed(0)
Lab4.events.clear()
assert(Lab4.makeArrivalEvent( 5000, 100 ) == None)
assert(Lab4.makeTellerEvent( 1000, 5 ) == None)
assert(Lab4.makeServiceEvent( 1000, 17, 900, 25 ) == None)
assert(len(Lab4.events) == 3)
assert(Lab4.getType(Lab4.events[0])=='A')
assert(Lab4.getTime(Lab4.events[0])==3155)
assert(Lab4.getTeller(Lab4.events[0])==99)
assert(Lab4.getWait(Lab4.events[0])==99999)
assert(Lab4.getService(Lab4.events[0])==194)
assert(Lab4.getType(Lab4.events[1])=='T')
assert(Lab4.getTime(Lab4.events[1])==1056)
assert(Lab4.getTeller(Lab4.events[1])==5)
assert(Lab4.getWait(Lab4.events[1])==56)
assert(Lab4.getService(Lab4.events[1])==99999)
assert(Lab4.getType(Lab4.events[2])=='S')
assert(Lab4.getTime(Lab4.events[2])==1025)
assert(Lab4.getTeller(Lab4.events[2])==17)
assert(Lab4.getWait(Lab4.events[2])==100)
assert(Lab4.getService(Lab4.events[2])==25)

<h3>The module variable *completed*</h3>

You will also need a module variable called *completed*.  You should put a line of code that initializes *completed* to be an empty list right below your statement that initializes *events.*

The purpose of *completed* is to store the list of service event strings for the customers who have completed their transactions and left the bank.  When the simulation begins this list starts empty, and as the simulation proceeds event strings are gradually appended to this list until there is an service event string for every customer.

<h3> 3) Analyzing the results of the simulation </h3>

Write a Python function called *analyzeResults* which takes no parameters.  The job of this funciton is to iterate through the event strings in *completed* and do two things:

1. Calculate the maximum wait time

2. Build a list of "visit times" (a customer's visit time is the wait time plus the service time).

After building this list, you should use the statistics module to calculate the mean and population standard deviation of the visit times.  

The function *analyzeResults* should return three values: maximum wait time, the mean visit time, and the population standard deviation.

In [4]:
# Test code for your function
Lab4.completed.clear()
Lab4.completed.append('01227S000000300209')
Lab4.completed.append('01618S000003300071')
Lab4.completed.append('01906S000002900161')
Lab4.completed.append('02018S000006500112')
Lab4.completed.append('02313S000001800069')
Lab4.completed.append('02913S000000700084')
Lab4.completed.append('03203S000003700249')
Lab4.completed.append('03412S000016300209')
Lab4.completed.append('03551S000014300139')
Lab4.completed.append('03563S010001700271')
assert(Lab4.analyzeResults() == (163, 208.9, 92.3552380755959))
Lab4.completed.clear()
Lab4.completed.append('01225S030000100209')
Lab4.completed.append('01591S000000600071')
Lab4.completed.append('01889S030001200161')
Lab4.completed.append('01953S020000000112')
Lab4.completed.append('02298S010000300069')
Lab4.completed.append('02913S010000700084')
Lab4.completed.append('03183S000001700249')
Lab4.completed.append('03249S020000000209')
Lab4.completed.append('03409S000000100139')
Lab4.completed.append('03548S010000200271')
assert(Lab4.analyzeResults() == (17, 162.3, 71.49832165862357))

<h3> The module variable *queues* </h3>

The final module variable you will need is a list called *queues*.  This list will actually store a *list of lists*. 

Your simulation program will offer the option of having single queue or a queue for each teller.  When using a single queue, the *queues* will continue a single list.  When using a queue for each teller, *queues* will contain a list for each teller.

<h3>4) Initializing the simulation </h3>

Write a Python function *initialize(nCustomers, nTellers, maxArrival, avgService, singleQueue, seed)* to set a random seed, initialize three module variables, and generate the initial events.  Follow these steps:

1.  Set the seed for the random number generator to *seed*

2.  Clear all three module variables

3.  Depending on whether *singleQueue* is True or False, either append [] to *queues* once or *nTellers* times.

4.  By calling makeArrivalEvent multiple times, add *nCustomers* random arrival events to the *events* list.

5.  By calling makeTellerEvent mutliple times, add *nTellers* random teller events to the *events* list.  The start time for each of these teller events should be 0.

In [5]:
# Test code for your function
assert(Lab4.initialize(10, 2, 3600, 138, True, 0)==None)
assert(len(Lab4.events) == 12)
assert(Lab4.queues == [[]])
assert(len(Lab4.completed) == 0)
assert(Lab4.events[0] =='03458A999999900197')
assert(Lab4.events[1] =='03104A999999900215')
assert(Lab4.events[2] =='00165A999999900132')
assert(Lab4.events[3] =='02094A999999900248')
assert(Lab4.events[4] =='01658A999999900155')
assert(Lab4.events[5] =='01952A999999900183')
assert(Lab4.events[6] =='02389A999999900111')
assert(Lab4.events[7] =='02067A999999900071')
assert(Lab4.events[8] =='01154A999999900071')
assert(Lab4.events[9] =='03095A999999900048')
assert(Lab4.events[10]=='00039T000003999999')
assert(Lab4.events[11]=='00051T010005199999')
assert(Lab4.initialize(5, 3, 3600, 138, False, 0) == None)
assert(len(Lab4.events) == 8)
assert(Lab4.queues == [[],[],[]])
assert(len(Lab4.completed) == 0)
assert(Lab4.events[0] =='03458A999999900197')
assert(Lab4.events[1] =='03104A999999900215')
assert(Lab4.events[2] =='00165A999999900132')
assert(Lab4.events[3] =='02094A999999900248')
assert(Lab4.events[4] =='01658A999999900155')
assert(Lab4.events[5] =='00030T000003099999')
assert(Lab4.events[6] =='00022T010002299999')
assert(Lab4.events[7] =='00037T020003799999')